In [1]:
import numpy as np
import math
import pandas as pd

# List of basic ship information

loa = 158 # ship overall length
lwl = 144.8 # ship waterline length, m
lpp = 0.97*lwl # ship perpendicular length , m, according to information
B = 24.5 # Ship B, m
depth = 13.8 # Ship depth. m
T_n = 5.85 # Nominal max draught , m
dwt = 5110 # ship dead weight , t
V_n = 17.7 # ship design speed, knots

#list of constants

g = 9.805 # gravity, kg/ms^2 
rho_sea = 1025 # kg/m3
nu_sea = 0.00000118 # Dynamic viscosity of sea m^2/s

In [2]:
v_dummy = [4,15,16,17.7,18,19]
v_dummy_df = pd.DataFrame(v_dummy,columns=['dummy_vel'])
v_dummy_df['dummy_vel_ms'] = v_dummy_df['dummy_vel'] / 1.94384

In [3]:
# Calculation for C_b according to Schneekluth and Bertram 1998
# first calculate froude number Fn
V_n = 17.7/1.94384
Fr_n = V_n / math.sqrt(g*lpp)
print(f"Froude Number {Fr_n:0.4f}")
C_b = -4.22 + 27.8*math.sqrt(Fr_n) - 39.1*Fr_n + 46.6*(Fr_n)**3
print(f"C_b {C_b:0.4f}")


Froude Number 0.2454
C_b 0.6451


In [4]:
# calculation for C_m according to charchalis 2017

C_m = 0.977 + 0.085*(C_b-0.6)
print(f"C_m {C_m:0.4f}")

# prismatic coefficient C_p can be calculated

C_p = C_b/C_m 
print(f"C_p {Fr_n:0.4f}")


C_m 0.9808
C_p 0.2454


In [5]:
# Displacement calculation according to barras
# Use approximate value cd=0.35 according to Barras

dsp = dwt/0.35 # m^3
print(f"dsp {dsp:0.4f} m^3")


dsp 14600.0000 m^3


In [6]:
# coefficient c14 to account for stern shape according to holtrop mennen

C_stern = 10 # assume u shaped stern
c14 = 1 + 0.011*C_stern 
print(f"c14 {c14:0.4f}")


c14 1.1100


In [7]:
# Calculate length of run according to holtrop mennen

lcb = -2/100 # ratio lcb wrt to amidship ford, value from barras
# L in holtrop mennen is lwl
lr = lwl*(1-C_p+(0.06*C_p*lcb/(4*C_p-1)))
print(f"lr {lr:0.4f} m")


lr 49.4890 m


In [8]:
# now the (1+k1) can be calculated

k1a = 0.487118*c14*(B/lwl)**1.06806
k1b = (T_n/lwl)**0.46106
k1c = (lwl/lr)**0.121563
k1d = (lwl**3/dsp)**0.36486
k1e = (1-C_p)**-0.604247

k1_const = 0.93 + k1a*k1b*k1c*k1d*k1e

print(f"k1_const {k1_const:0.4f}")


k1_const 1.2119


In [9]:
# we can then calculate R_f
import pandas as pd
import numpy as np
# use v mean = 10 for Reynold


v_dummy_df['Re'] =( v_dummy_df['dummy_vel_ms'] * lwl ) / nu_sea
v_dummy_df['C_f'] = 0.075 / (np.log10(v_dummy_df['Re']-2)**2)
print(v_dummy_df['Re'])
print(v_dummy_df['C_f'])

# calculate appendage area of bare hull S_bh

# first calculate C_wp acc. to schneekluth and bertram
C_wp = (1+2*C_b)/3
print(f"C_wp {C_wp:0.4f}")

# A_bt according to kim

A_m = B*T_n*C_m
A_t = 0.051 * A_m
A_bt = 0.08*A_m
print(A_bt)

print(f"A_bt {A_bt:0.4f} m^2")

sbh_a = lwl*(2*T_n+B)*math.sqrt(C_m)
sbh_b = 0.453
sbh_c = 0.4425*C_b
sbh_d = 0.2862*C_m
sbh_e = 0.003467*(B/T_n)
sbh_f = 0.3696*C_wp
sbh_g = 2.38*A_bt/C_b

S_bh = sbh_a*(sbh_b+sbh_c-sbh_d+sbh_e+sbh_f)+sbh_g
print(f"S_bh {S_bh:0.4f} m^2")

v_dummy_df['R_f'] = 0.5 * rho_sea * (v_dummy_df['dummy_vel_ms'])**2 * v_dummy_df['C_f'] * S_bh * k1_const
print(v_dummy_df['R_f'])



0    2.525143e+08
1    9.469287e+08
2    1.010057e+09
3    1.117376e+09
4    1.136314e+09
5    1.199443e+09
Name: Re, dtype: float64
0    0.001062
1    0.000931
2    0.000925
3    0.000916
4    0.000915
5    0.000910
Name: C_f, dtype: float64
C_wp 0.7634
11.24627236597615
A_bt 11.2463 m^2
S_bh 3958.0019 m^2
0     11058.515982
1    136256.715808
2    154066.211728
3    186721.361999
4    192793.359170
5    213700.189821
Name: R_f, dtype: float64


In [10]:
# Now resistance due to appendage
# Assume Sapp

S_app = 50 # m^2 
# from holtrop mennen
k2_const = 2.8

v_dummy_df['R_app'] = 0.5 * rho_sea * (v_dummy_df['dummy_vel_ms'])**2 * (v_dummy_df['C_f']) * S_app *k2_const
print(v_dummy_df['R_app'])


0     322.765568
1    3976.932922
2    4496.739746
3    5449.847572
4    5627.071317
5    6237.280235
Name: R_app, dtype: float64


In [11]:
print(B/lwl)
print(lwl/B)
print(lwl**3/dsp)

0.1691988950276243
5.910204081632654
207.94708164383565


In [12]:
# calculate wave resistance
c7 = B/lwl
T_fwd = T_n # assume !
h_b = 0.6*T_fwd
c3 = 0.56 * A_bt**1.5 / (B*T_n*(0.31*math.sqrt(A_bt)+T_fwd-h_b))
print(c3)
c2 = math.exp(-1.89*math.sqrt(c3))
print(c2)
c5 = 1 - 0.8*(A_t/(B*T_n*C_m))
print(c5)
lambda_const = (1.446 * C_p) - 0.03*(lwl/B)
print(lambda_const)
c16 = 8.07981*C_p - 13.8673*C_p**2 + 6.984388*C_p**3
print(c16)
m_1 = 0.0140407 * (lwl/T_n) - 1.75254*(dsp**(1/3)/lwl) -  4.79323*(B/lwl) - c16
print(m_1)
c15 = -1.69385
# Use dynamic Froude here
v_dummy_df['Fr_n'] = v_dummy_df['dummy_vel_ms'] / math.sqrt(g*lpp)
v_dummy_df['m_2'] = c15 * C_p**2 * np.exp(-0.1*v_dummy_df['Fr_n']**-2)
print(v_dummy_df['m_2'])
i_e = 1 + 89*math.exp(-(lwl/B)**0.80856*(1-C_wp)**0.30484*(1-C_p-0.0225*lcb)**0.6367*(lr/B)**0.34574*((100*dsp)/lwl**3)**0.16302)
print(i_e)
c1 = 2223105 * c7**3.78613 * (T_n/B)**1.07961*(90-i_e)**-1.37565
print(c1)
d = -0.9

v_dummy_df['R_w'] = c1*c2*c5*dsp*g*rho_sea*np.exp(m_1*v_dummy_df['Fr_n']**d+v_dummy_df['m_2']*np.cos(lambda_const*v_dummy_df['Fr_n']**-2))
print(v_dummy_df['R_w'])

0.04360273752148408
0.6739118418119685
0.9592
0.7737872716316899
1.302542119617646
-2.061826835604456
0   -5.501476e-15
1   -7.253964e-02
2   -9.598494e-02
3   -1.391982e-01
4   -1.470545e-01
5   -1.733676e-01
Name: m_2, dtype: float64
19.867152251684907
1.6394657947130131
0         0.000126
1     31126.295614
2     57638.632784
3     91773.557081
4    101174.087271
5    160252.169767
Name: R_w, dtype: float64


In [13]:
print(A_bt)
print(T_fwd)

11.24627236597615
5.85


In [14]:
# resistance due to bulbous bow
# use V = 10 m/s

P_b = 0.56*math.sqrt(A_bt)/(T_fwd-1.5*h_b)
print(P_b)
v_dummy_df['F_ni'] = v_dummy_df['dummy_vel_ms'] / np.sqrt(g*(T_fwd-h_b-0.25*math.sqrt(A_bt))+0.15*v_dummy_df['dummy_vel_ms']**2)
v_dummy_df['R_b'] = 0.11 * np.exp(-3*P_b**-2)*v_dummy_df['F_ni']**3*A_bt**1.5*rho_sea*g/(1+v_dummy_df['F_ni']**2)
print(v_dummy_df['R_b'])

3.210232295573174
0     3536.521258
1    35386.872486
2    37606.727441
3    41014.586372
4    41570.874930
5    43333.796507
Name: R_b, dtype: float64


In [15]:
#Calculate Transom Resistance 
# use v = 10

F_nT = 10 / math.sqrt(2*g*A_t/(B+B*C_wp))
print(F_nT)
c6 = 0
R_tr = 0.5*rho_sea*10**2*A_t*c6
print(f"R_tr = {R_tr:0.4f} N")

5.5434260311174
R_tr = 0.0000 N


In [16]:
# Model ship correlation resistance

c4 = T_fwd/lwl
print(c4)
# since c4>0.04, use c4 as c4 = 0.04
c4 = 0.04
C_a = 0.006*(lwl+100)**-0.16 - 0.00205 + 0.003*math.sqrt(lwl/7.5)*C_b**4*c2*(0.04-c4)
print(C_a)
v_dummy_df['R_a'] = 0.5*rho_sea*v_dummy_df['dummy_vel_ms']**2*S_bh*C_a
print(v_dummy_df['R_a'])


0.04040055248618784
0.00043852165454709695
0     3766.689918
1    52969.076972
2    60267.038689
3    73754.142776
4    76275.470840
5    84985.941276
Name: R_a, dtype: float64


In [17]:
# Calculate Rtot

v_dummy_df['R_tot'] = v_dummy_df['R_f'] + v_dummy_df['R_app'] + v_dummy_df['R_w'] + v_dummy_df['R_b'] + v_dummy_df['R_a'] 

In [18]:
v_dummy_df.head()

,dummy_vel,dummy_vel_ms,Re,C_f,R_f,R_app,Fr_n,m_2,R_w,F_ni,R_b,R_a,R_tot
0,4.0,2.057783,2.525143e+08,0.001062,11058.515982,322.765568,0.055451,-5.501476e-15,0.000126,0.525080,3536.521258,3766.689918,18684.492851
1,15.0,7.716685,9.469287e+08,0.000931,136256.715808,3976.932922,0.207939,-7.253964e-02,31126.295614,1.586593,35386.872486,52969.076972,259715.893803
2,16.0,8.231130,1.010057e+09,0.000925,154066.211728,4496.739746,0.221802,-9.598494e-02,57638.632784,1.649991,37606.727441,60267.038689,314075.350387
3,17.7,9.105688,1.117376e+09,0.000916,186721.361999,5449.847572,0.245368,-1.391982e-01,91773.557081,1.747210,41014.586372,73754.142776,398713.495800
4,18.0,9.260021,1.136314e+09,0.000915,192793.359170,5627.071317,0.249527,-1.470545e-01,101174.087271,1.763078,41570.874930,76275.470840,417440.863529


In [19]:
v_dummy_df2=v_dummy_df.drop(['C_f','Re','F_ni'],axis=1)

In [20]:
# Calculate Efficiencies

# First n_h
# Approximate diameter size from Bertram

D = 0.215*16 #m
# D = 8
print(D)
print(C_b)
print(k1_const)
print(B)
print(T_n)

v_dummy_df['C_v'] = k1_const*v_dummy_df['C_f'] + C_a
# print(v_dummy_df['C_v'])
v_dummy_df['w'] = 0.3095 * C_b + 10*v_dummy_df['C_v']*C_b - (0.23*D)/math.sqrt(B*T_n) 
# print(v_dummy_df['w'])
v_dummy_df['t'] = 0.325*C_b - 0.1885*D/math.sqrt(B*T_n)
# print(v_dummy_df['t'])
v_dummy_df['eff_h'] = (1-v_dummy_df['t']) / (1-v_dummy_df['w'])
print(v_dummy_df['eff_h'])

# Then n_r
v_dummy_df['eff_r'] = 0.9737 + 0.111*(C_p - 0.225*lcb) - 0.06325*lcb
print(v_dummy_df['eff_r'])

v_dummy_df['eff_s'] = 0.99 # Set according to holtrop mennen and man
v_dummy_df['eff_o'] = 0.75 # Approx wageningen kristensen

v_dummy_df2['eff_tot'] = v_dummy_df['eff_h']* v_dummy_df['eff_r']* v_dummy_df['eff_s']*v_dummy_df['eff_o']
# print(v_dummy_df2)
v_dummy_df2['P_b'] = (v_dummy_df2['R_tot']/1e3 * v_dummy_df2['dummy_vel_ms'])/(v_dummy_df2['eff_tot']*0.85) # in kW
v_dummy_df2.head()
# Unit P_b is KW

3.44
0.6451363786295532
1.2118857645707746
24.5
5.85
0    0.987385
1    0.986199
2    0.986147
3    0.986067
4    0.986053
5    0.986011
Name: eff_h, dtype: float64
0    1.048474
1    1.048474
2    1.048474
3    1.048474
4    1.048474
5    1.048474
Name: eff_r, dtype: float64


,dummy_vel,dummy_vel_ms,R_f,R_app,Fr_n,m_2,R_w,R_b,R_a,R_tot,eff_tot,P_b
0,4.0,2.057783,11058.515982,322.765568,0.055451,-5.501476e-15,0.000126,3536.521258,3766.689918,18684.492851,0.768671,50.019606
1,15.0,7.716685,136256.715808,3976.932922,0.207939,-7.253964e-02,31126.295614,35386.872486,52969.076972,259715.893803,0.767748,2610.421250
2,16.0,8.231130,154066.211728,4496.739746,0.221802,-9.598494e-02,57638.632784,37606.727441,60267.038689,314075.350387,0.767707,3367.422321
3,17.7,9.105688,186721.361999,5449.847572,0.245368,-1.391982e-01,91773.557081,41014.586372,73754.142776,398713.495800,0.767645,4729.480269
4,18.0,9.260021,192793.359170,5627.071317,0.249527,-1.470545e-01,101174.087271,41570.874930,76275.470840,417440.863529,0.767634,5035.615174


In [21]:
SFOC = 169.4 # g/kWh
v_dummy_df2['FOC'] = (v_dummy_df2['P_b'] * SFOC)/1e6 # get FOC t/h
v_dummy_df2.head()


,dummy_vel,dummy_vel_ms,R_f,R_app,Fr_n,m_2,R_w,R_b,R_a,R_tot,eff_tot,P_b,FOC
0,4.0,2.057783,11058.515982,322.765568,0.055451,-5.501476e-15,0.000126,3536.521258,3766.689918,18684.492851,0.768671,50.019606,0.008473
1,15.0,7.716685,136256.715808,3976.932922,0.207939,-7.253964e-02,31126.295614,35386.872486,52969.076972,259715.893803,0.767748,2610.421250,0.442205
2,16.0,8.231130,154066.211728,4496.739746,0.221802,-9.598494e-02,57638.632784,37606.727441,60267.038689,314075.350387,0.767707,3367.422321,0.570441
3,17.7,9.105688,186721.361999,5449.847572,0.245368,-1.391982e-01,91773.557081,41014.586372,73754.142776,398713.495800,0.767645,4729.480269,0.801174
4,18.0,9.260021,192793.359170,5627.071317,0.249527,-1.470545e-01,101174.087271,41570.874930,76275.470840,417440.863529,0.767634,5035.615174,0.853033
